In [ ]:
# Chainer 
import numpy as np
import chainer
import chainer.links as L
import chainer.functions as F
from chainer import optimizers

x_array = np.random.random_sample((1,784)).astype(np.float32)
t_array = np.asarray([[1,0,0,0,0,0,0,0,0,0]]).astype(np.float32)
x = chainer.Variable(x_array)
t = chainer.Variable(t_array)
x.to_gpu()
t.to_gpu()

class MLP(chainer.Chain):
    def __init__(self):
        super(MLP, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, 1000)  
            self.l2 = L.Linear(None, 1000)  
            self.l3 = L.Linear(None, 10)  

    def forward(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

model = MLP()
model.to_gpu()
optimizer = optimizers.SGD(lr=0.01)
optimizer.setup(model)

for i in range(100):
    y = model(x)
    loss = F.mean_squared_error(y, t)
    model.cleargrads()
    loss.backward()
    optimizer.update()

y.to_cpu()
y_array = y.data
print(y_array)

In [ ]:
# PyTorch
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x_array = np.random.random_sample((1,784)).astype(np.float32)
t_array = np.asarray([[1,0,0,0,0,0,0,0,0,0]]).astype(np.float32)
x = torch.tensor(x_array)
t = torch.tensor(t_array)
x = x.to("cuda")
t = t.to("cuda")

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(784, 1000)  
        self.l2 = nn.Linear(1000, 1000)  
        self.l3 = nn.Linear(1000, 10) 
        
    def forward(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)
    
model = MLP()
model = model.to("cuda")
optimizer = optim.SGD(model.parameters(), lr=0.01)

for i in range(100):
    y = model(x)
    loss = F.mse_loss(y, t)
    model.zero_grad()
    loss.backward()
    optimizer.step()

y = y.to("cpu")
y_array = y.detach().numpy()
print(y_array)